In [1]:
import joblib
from multiprocessing import cpu_count
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import os
from scipy.stats import uniform, loguniform, randint    
from xgboost import XGBClassifier

from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import f1_score
from numpy import linspace

In [2]:
# Caminho
data_dir = "C:/Users/Elaine/Desktop/TCC_acidentes/data/" 

# Carregar o preprocess
preprocess = joblib.load('preprocess.pkl')

# Carregar os splits
X_train, X_test, y_train, y_test = joblib.load('data_splits.pkl')

X_train.info()
X_train.head()

c:\Users\Elaine\Desktop\TCC_acidentes\venv\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.7.2 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Elaine\Desktop\TCC_acidentes\venv\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.2 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Elaine\Desktop\TCC_acidentes\venv\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.7.2 when u

<class 'pandas.DataFrame'>
Index: 91628 entries, 44026 to 43353
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uf                      91628 non-null  object 
 1   br                      91433 non-null  float64
 2   km                      91433 non-null  float64
 3   dia_semana              91628 non-null  object 
 4   fase_dia                91628 non-null  object 
 5   sentido_via             91628 non-null  object 
 6   condicao_metereologica  91628 non-null  object 
 7   tipo_pista              91628 non-null  object 
 8   tracado_via             91628 non-null  object 
 9   uso_solo                91628 non-null  object 
 10  tipo_veiculo            88174 non-null  object 
 11  ano                     91628 non-null  int64  
 12  frota                   90614 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 9.8+ MB


,uf,br,km,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,ano,frota
44026,GO,40.0,16.0,sÃ¡bado,Plena Noite,Crescente,CÃ©u Claro,Dupla,Reta,Sim,AutomÃ³vel,2019,66560.0
9134,PE,101.0,81.0,terÃ§a-feira,Pleno dia,Crescente,CÃ©u Claro,MÃºltipla,Retorno Regulamentado,NÃ£o,Semireboque,2021,217949.0
54794,PR,476.0,135.0,quarta-feira,Pleno dia,Decrescente,CÃ©u Claro,MÃºltipla,Reta,Sim,AutomÃ³vel,2020,1622891.0
106025,SC,101.0,311.0,quarta-feira,Plena Noite,Crescente,Nublado,Simples,RotatÃ³ria,NÃ£o,AutomÃ³vel,2021,31357.0
52102,MS,163.0,490.0,quinta-feira,Plena Noite,Crescente,CÃ©u Claro,Simples,RotatÃ³ria,Sim,Ãnibus,2019,613060.0


In [4]:
# Variáveis Categóricas
cat_features = ['br', 'uf', 'dia_semana', 'fase_dia', 'sentido_via', 
                'condicao_metereologica', 'tipo_pista', 'tracado_via', 
                'uso_solo', 'tipo_veiculo'
                ] 

for col in cat_features:
    X_train[col] = X_train[col].astype(str).astype('category')
    X_test[col] = X_test[col].astype(str).astype('category')
    valid_categories = X_train[col].cat.categories
    X_test[col] = X_test[col].astype(str).astype('category')
    X_test[col] = X_test[col].cat.set_categories(valid_categories)


# Modelo
xgb = XGBClassifier(
    tree_method='hist',      
    enable_categorical=True,  #! No lugar de One Hot Encoder
    objective='binary:logistic',
    n_jobs=cpu_count() // 2,
    random_state=42
)

# Grid
param_grid = {
    'n_estimators': randint(500, 1000),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(4, 10),
    'scale_pos_weight': uniform(2, 6), # Ajuste pro seu desbalanceamento
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4)
}

# Busca
random_xgb = RandomizedSearchCV(
    xgb,
    param_grid,
    n_iter=50,
    scoring='f1',
    cv=5,
    verbose=1,
    n_jobs=cpu_count() // 2,
)

In [5]:
print(X_train.dtypes)


uf                        category
br                        category
km                         float64
dia_semana                category
fase_dia                  category
sentido_via               category
condicao_metereologica    category
tipo_pista                category
tracado_via               category
uso_solo                  category
tipo_veiculo              category
ano                          int64
frota                      float64
dtype: object


In [6]:
import xgboost
import sklearn
print("X_train é:", type(X_train))
print("versão xgb:", xgboost.__version__)
print("Versão sk:", sklearn.__version__)


X_train é: <class 'pandas.DataFrame'>
versão xgb: 3.2.0
Versão sk: 1.8.0


In [7]:
# Fit
random_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'colsample_bytree': <scipy.stats....0025DB0256060>, 'learning_rate': <scipy.stats....0025DB0331A90>, 'max_depth': <scipy.stats....0025DF33B3390>, 'n_estimators': <scipy.stats....0025DB0331940>, ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",50
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'f1'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",2
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` f

In [8]:
# Resultados
y_pred_xgb = random_xgb.predict(X_test)
print(f"Melhor F1 na Validação Cruzada: {random_xgb.best_score_:.4f}")
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

Melhor F1 na Validação Cruzada: 0.4932
              precision    recall  f1-score   support

           0       0.91      0.91      0.91     19305
           1       0.52      0.53      0.53      3603

    accuracy                           0.85     22908
   macro avg       0.72      0.72      0.72     22908
weighted avg       0.85      0.85      0.85     22908

[[17569  1736]
 [ 1702  1901]]


In [9]:
artifacts = {
    "model": random_xgb.best_estimator_,
    "categories": {
        col: X_train[col].cat.categories.tolist()
        for col in cat_features
    },
    "feature_order": X_train.columns.tolist()
}


In [10]:
joblib.dump(artifacts, "C:/Users/Elaine/Desktop/TCC_acidentes/models/xgb_artifacts.pkl")

['C:/Users/Elaine/Desktop/TCC_acidentes/models/xgb_artifacts.pkl']